<a href="https://colab.research.google.com/github/saikrithik/aicrowd-insurance-pricing-game/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!bash <(curl -sL https://gitlab.aicrowd.com/jyotish/pricing-game-notebook-scripts/raw/master/python/setup.sh)
from aicrowd_helpers import *

⚙️ Installing AIcrowd utilities...
  Running command git clone -q https://gitlab.aicrowd.com/yoogottamk/aicrowd-cli /tmp/pip-req-build-emifkirz
✅ Installed AIcrowd utilities


In [69]:
import sklearn

class Config:
  TRAINING_DATA_PATH = 'training.csv'
  MODEL_OUTPUT_PATH = 'model.pkl'
  AICROWD_API_KEY = 'a1dde7ec8733613146ca350a8999d4fb'  # You can get the key from https://aicrowd.com/participants/me
  APT_PACKAGES = [
    "libgomp1",
  ]
  ADDITIONAL_PACKAGES = [
    'numpy',  # you can define versions as well, numpy==0.19.2
    'pandas',
    'scikit-learn==' + sklearn.__version__,
    'lightgbm',
    'joblib'
  ]

In [70]:
%download_aicrowd_dataset

💾 Downloading dataset...
Verifying API Key...
API Key valid
Saved API Key successfully!
✅ Downloaded dataset


In [71]:
!pip install reg_resampler

In [88]:
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
import joblib
from reg_resampler import resampler
import warnings
warnings.filterwarnings("ignore")

In [73]:
df_train = pd.read_csv(Config.TRAINING_DATA_PATH)
x = df_train.drop(columns=['claim_amount'])
y = df_train['claim_amount']
x_raw = x.copy()

In [74]:
str_cols = [ col  for col, dt in df_train.dtypes.items() if dt == object]
num_cols = [x for x in df_train.columns.to_list() if x not in str_cols]

In [75]:
null_frame = pd.DataFrame(df_train.isnull().sum(),columns=["Value"])
display(null_frame[null_frame["Value"]>0])

,Value
drv_age2,152896
drv_age_lic2,152896
vh_age,4
vh_speed,2552
vh_value,2552
vh_weight,2552


In [76]:
def preprocess(X_raw):
  le = LabelEncoder()
  temp = X_raw.copy()
  for col in temp.columns:
    if temp[col].dtype == object:
      temp[col] = le.fit_transform(temp[col])
  # if 'vh_make_model' in temp.columns:
  #   temp = temp.drop(['vh_make_model'], axis = 1)
  return temp

In [77]:
df_train = preprocess(df_train)
x = preprocess(x_raw)

In [78]:
temp = y**2
temp.mean()

538841.8799178002

In [79]:
114.18**2

13037.072400000001

In [80]:
y

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
228211    0.0
228212    0.0
228213    0.0
228214    0.0
228215    0.0
Name: claim_amount, Length: 228216, dtype: float64

In [81]:
x

,id_policy,year,pol_no_claims_discount,pol_coverage,pol_duration,pol_sit_duration,pol_pay_freq,pol_payd,pol_usage,drv_sex1,drv_age1,drv_age_lic1,drv_drv2,drv_sex2,drv_age2,drv_age_lic2,vh_make_model,vh_age,vh_fuel,vh_type,vh_speed,vh_value,vh_weight,population,town_surface_area
0,0,1.0,0.332,2,5,1,1,0,3,1,35.0,16.0,1,1,26.0,1.0,33,8.0,1,1,174.0,11040.0,1143.0,1270.0,33.1
1,25382,1.0,0.000,2,6,1,1,0,3,1,60.0,41.0,0,0,NaN,NaN,33,10.0,0,1,174.0,11040.0,1143.0,1290.0,51.3
2,25383,1.0,0.196,1,2,1,3,1,2,1,55.0,35.0,1,1,57.0,38.0,341,8.0,0,0,150.0,14159.0,1193.0,1020.0,262.8
3,25384,1.0,0.000,2,8,5,3,0,3,0,54.0,31.0,0,0,NaN,NaN,416,4.0,1,1,149.0,17233.0,1012.0,180.0,219.7
4,25385,1.0,0.000,1,2,2,3,0,2,0,65.0,38.0,0,0,NaN,NaN,709,13.0,1,1,200.0,19422.0,1315.0,30.0,70.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228211,5240,4.0,0.087,2,4,4,1,0,3,0,50.0,28.0,1,1,23.0,5.0,750,15.0,0,1,179.0,16047.0,955.0,1030.0,95.3
228212,32820,4.0,0.000,0,14,5,1,0,3,1,40.0,22.0,1,1,34.0,12.0,243,5.0,0,1,139.0,11985.0,1264.0,30.0,89.6
228213,36771,4.0,0.181,0,10,4,3,0,2,0,71.0,37.0,0,0,NaN,NaN,693,15.0,1,1,175.0,23900.0,1285.0,290.0,106.1
228214,36347,4.0,0.000,0,25,15,0,0,1,1,49.0,31.0,0,0,NaN,NaN,578,31.0,0,1,198.0,20002.0,1320.0,30.0,104.4


In [82]:
rs = resampler()
rs.fit(preprocess(df_train),target="claim_amount",bins=100)

INFO: Class 9 has been merged into Class 8 due to low number of samples
INFO: Class 10 has been merged into Class 8 due to low number of samples
INFO: Class 11 has been merged into Class 8 due to low number of samples
INFO: Class 12 has been merged into Class 8 due to low number of samples
INFO: Class 13 has been merged into Class 8 due to low number of samples
INFO: Class 14 has been merged into Class 8 due to low number of samples
INFO: Class 15 has been merged into Class 8 due to low number of samples
INFO: Class 16 has been merged into Class 8 due to low number of samples
INFO: Class 17 has been merged into Class 8 due to low number of samples
INFO: Class 18 has been merged into Class 8 due to low number of samples
INFO: Class 19 has been merged into Class 8 due to low number of samples
INFO: Class 20 has been merged into Class 8 due to low number of samples
INFO: Class 21 has been merged into Class 8 due to low number of samples
INFO: Class 22 has been merged into Class 8 due to l

array([0, 0, 0, ..., 0, 0, 0])

In [123]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
rs = resampler()
# Generate classes
Y_classes = rs.fit(preprocess(df_train),target="claim_amount",bins=100)
# Create the actual target variable
Y = df_train["claim_amount"]
# Perform K-Fold
kfold, scores = KFold(n_splits=10, shuffle=True, random_state=27), list()
for train, test in kfold.split(df_train):
# kfold, scores = StratifiedKFold(n_splits=5, shuffle=True, random_state=27), list()
# for train, test in kfold.split(df_train, Y_classes):
    
    # Split into train and test
    x_train, y_train = df_train.iloc[train], Y_classes[train]
    x_test, y_test = df_train.iloc[test], Y.iloc[test]
    # Remove the target variable from x_test
    x_test = x_test.drop("claim_amount", axis=1)
    # Get the class distriubtion for perfoming relative sampling in the next line
    xp = Counter(y_train)
    #Under sampling
    rus = RandomUnderSampler(random_state=27, sampling_strategy={0:int(xp[0]*0.03)})
    x_train, y_train = rs.resample(rus, x_train.fillna(-999), y_train)
    # Your favourite oversampler
    # smote = SMOTE(random_state=27, sampling_strategy={8:int(xp[8]*100.0), 7: int(xp[7]*200.0),6: int(xp[6]*205.0),
    #                                                   5: int(xp[5]*100.2)})
    # smote = SMOTE(random_state=27, sampling_strategy={
    #                                                   5: int(xp[5]*1)})
    # Generate the over-sampled data
    # x_train, y_train = rs.resample(smote, x_train.fillna(-999), y_train)
    # display(x_train)
    # Fit the model
    model = LGBMRegressor(random_state=27,n_estimators=100)
    model.fit(x_train.values, np.log(y_train.values+5))
    preds = np.exp(model.predict(x_test.values))
    
    # Check the score
    score = np.sqrt(mean_squared_error(y_test.values, preds))
    print(score)
    scores.append(score)
print("\nAverage: ", sum(scores)/len(scores))

INFO: Class 9 has been merged into Class 8 due to low number of samples
INFO: Class 10 has been merged into Class 8 due to low number of samples
INFO: Class 11 has been merged into Class 8 due to low number of samples
INFO: Class 12 has been merged into Class 8 due to low number of samples
INFO: Class 13 has been merged into Class 8 due to low number of samples
INFO: Class 14 has been merged into Class 8 due to low number of samples
INFO: Class 15 has been merged into Class 8 due to low number of samples
INFO: Class 16 has been merged into Class 8 due to low number of samples
INFO: Class 17 has been merged into Class 8 due to low number of samples
INFO: Class 18 has been merged into Class 8 due to low number of samples
INFO: Class 19 has been merged into Class 8 due to low number of samples
INFO: Class 20 has been merged into Class 8 due to low number of samples
INFO: Class 21 has been merged into Class 8 due to low number of samples
INFO: Class 22 has been merged into Class 8 due to l

In [99]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import numpy as np
splits = 9
kfold, scores = KFold(n_splits=splits,shuffle=True, random_state=27), []
# oof = []
for train_, test_ in kfold.split(x,y):
    x_train, x_test = x.iloc[train_], x.iloc[test_]
    y_train, y_test = y[train_], y[test_]
    model = LGBMRegressor(random_state=27,n_estimators = 4)
    model.fit(x_train.values, np.log(y_train.values+0.01))
    preds = np.exp(model.predict(x_test.values))
    
    # Check the score
    score = np.sqrt(mean_squared_error(y_test.values, preds))
    print(score)
    scores.append(score)
    # oof.append(model.predict_proba(df_test))
print("Average:Score", sum(scores)/len(scores))

545.2344910814967
529.9700131062526
575.8737597915418
1082.534522757594
779.9995728525672
575.4053621312625
760.1035442782924
994.956894744807
510.5749063314361
Average:Score 706.0725630083612
